# Add Wikipedia title phrases

This notebook searches for adjacent, original and lemmatized tokens (5-grams) which are equal to lemmatized Wikipedia titles of length 2-5.

It replaces these tokens with a single, concatenated token.

This processing step is already applied when running the phrase_extraction_in_batches.py script. Apply this notebook only on files that are missing this step.

In [4]:
# coding: utf-8

from constants import *
from os import listdir
from os.path import isfile, join
import gc
import pandas as pd
import re
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_rows = 2001

In [5]:
pattern = re.compile(r'FO')
files = sorted([
    f for f in listdir(SMPL_PATH)
    if (isfile(join(SMPL_PATH, f)) and pattern.match(f))
])

for name in files[:]:
    gc.collect()
    corpus = name.split('.')[0]
    print(corpus)
    f = join(SMPL_PATH, f'{corpus}.pickle')
    df = pd.read_pickle(f)

    # filter Wikipedia documents. Process only valid articles. Might be redundant.
    if name.startswith('dewiki'):
        if goodids is None:
            goodids = pd.read_pickle(join(ETL_PATH, 'dewiki_good_ids.pickle'))
        df = df[df.hash.isin(goodids.index)]

    # fixes wrong POS tagging for punctuation. Might be redundant.
    mask_punct = df[TOKEN].isin(list('[]<>/–%'))
    df.loc[mask_punct, POS] = PUNCT

    df = insert_wikipedia_phrases(df)
    f = join(SMPL_PATH, 'wiki_phrases', f'{corpus}_wiki_phrases.pickle')
    print(f'Writing {f}')
    df.to_pickle(f)
    gc.collect()

FOCUS_cleansed_simple


100%|██████████| 24652775/24652775 [31:30<00:00, 13042.51it/s]


Writing ../data/preprocessed/simple/wiki_phrases/FOCUS_cleansed_simple_wiki_phrases.pickle
